In [2]:
import pandas as pd

# Import Data / Transform into Dataframes

In [3]:
users_df = pd.read_csv('data/users.dat', delimiter='::', engine='python', names=['uid', 'gender', 'age', 'occupation', 'zip'])
movies_df = pd.read_csv('data/movies.dat', delimiter='::', engine='python', names=['movie_id', 'title', 'genre'], encoding='latin-1')
ratings_df = pd.read_csv('data/ratings.dat', delimiter='::', engine='python', names=['uid', 'movie_id', 'rating', 'ts'])

In [4]:
users_df.head()

,uid,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
movies_df.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings_df.head()

,uid,movie_id,rating,ts
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# Join Dataframes

In [17]:
movie_ratings_df = ratings_df.join(movies_df, on='movie_id', lsuffix='_ratings', rsuffix='_movies')
movie_ratings_df.drop(['movie_id_ratings', 'movie_id_movies'], inplace=True, axis=1)
movie_ratings = movie_ratings_df.join(users_df, on='uid', lsuffix='_movies', rsuffix='_users')
movie_ratings.drop(['movie_id_ratings', 'movie_id_movies'], inplace=True, axis=1)


,movie_id,uid,rating,ts,title,genre
0,1193,1,5,978300760,Wings of Desire (Der Himmel über Berlin) (1987),Comedy|Drama|Romance
1,661,1,3,978302109,Bloodsport 2 (1995),Action
2,914,1,3,978301968,All About Eve (1950),Drama
3,3408,1,4,978300275,Empire Records (1995),Comedy|Drama
4,2355,1,5,978824291,You've Got Mail (1998),Comedy|Romance
